In [19]:
from typing import Optional, List
from bs4 import BeautifulSoup, Tag
import requests

def get_realtime_price(url: str = 'https://finance.naver.com/sise/sise_index.naver?code=KOSDAQ') :
    req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    dom = BeautifulSoup(req.text, "html.parser")
    cur_price = dom.select_one('em#now_value').text
    cur_volume = dom.select_one('td#quant').text
    cur_volume_price = dom.select_one('td#amount').text
    cur_volume_personal = dom.select_one('dl.lst_kos_info').select_one(':nth-child(2)').select_one('span').text
    cur_volume_forg = dom.select_one('dl.lst_kos_info').select_one(':nth-child(3)').select_one('span').text
    cur_volume_instfund = dom.select_one('dl.lst_kos_info').select_one(':nth-child(4)').select_one('span').text

    cur_volume_prog_pos = dom.select_one('dl.lst_kos_info').select_one(':nth-child(6)').select_one('span').text
    cur_volume_prog_neg = dom.select_one('dl.lst_kos_info').select_one(':nth-child(7)').select_one('span').text
    
    print(cur_volume_personal + '|' + cur_volume_forg + '|' + cur_volume_instfund + '|' 
          + cur_volume_prog_pos + '|' + cur_volume_prog_neg)
    print(cur_price + "|" + cur_volume + "|" + cur_volume_price + "|" )
    

get_realtime_price()
    

+1,394억|-1,130억|-134억|-12억|-1,099억
777.47|1,135,628|7,653,033|


In [58]:
# 코스닥 종가
from typing import Optional, List
from bs4 import BeautifulSoup, Tag
import requests
from dataclasses import dataclass

@dataclass
class RealtimePriceCrawl:
    date: str
    closing_price: float
    up_down_per: float
    volume: int
    volume_amt: int

def get_kosdaq_daily_price() :
    url = 'https://finance.naver.com/sise/sise_index_day.naver?code=KOSDAQ&page=1'
    req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    dom = BeautifulSoup(req.text, "html.parser")
    
    v_date = dom.select('tr > td.date')
    
    res: List[RealtimePriceCrawl] = []
    for v in v_date:
        print('---------------------------------------')
        print(v.parent.select_one(':nth-child(1)').text)
        print(v.parent.select_one(':nth-child(2)').text)
        print(v.parent.select_one(':nth-child(4)').text.strip())
        print(v.parent.select_one(':nth-child(5)').text)
        print(v.parent.select_one(':nth-child(6)').text)
        # res.append(RealtimePriceCrawl(
        #     date=v.parent.select_one(':nth-child(1)').text, 
        #     closing_price=v.parent.select_one(':nth-child(2)').text))
    
get_kosdaq_daily_price()

---------------------------------------
2024.08.19
777.47
-1.13%
1,135,628
7,653,033
---------------------------------------
2024.08.16
786.33
+1.22%
1,080,402
8,000,416
---------------------------------------
2024.08.14
776.83
+1.56%
1,111,020
7,691,492
---------------------------------------
2024.08.13
764.86
-1.02%
1,170,514
7,661,966
---------------------------------------
2024.08.12
772.72
+1.08%
885,108
6,850,312
---------------------------------------
2024.08.09
764.43
+2.57%
843,852
7,392,825
